# Collaboration and Competition

---

In this notebook, you will learn how to use the Unity ML-Agents environment for the third project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893) program.

### 1. Start the Environment

We begin by importing the necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [1]:
from unityagents import UnityEnvironment
import numpy as np
import torch
from agent import Agent
from collections import deque
import time
import matplotlib.pyplot as plt
%matplotlib inline

Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Tennis.app"`
- **Windows** (x86): `"path/to/Tennis_Windows_x86/Tennis.exe"`
- **Windows** (x86_64): `"path/to/Tennis_Windows_x86_64/Tennis.exe"`
- **Linux** (x86): `"path/to/Tennis_Linux/Tennis.x86"`
- **Linux** (x86_64): `"path/to/Tennis_Linux/Tennis.x86_64"`
- **Linux** (x86, headless): `"path/to/Tennis_Linux_NoVis/Tennis.x86"`
- **Linux** (x86_64, headless): `"path/to/Tennis_Linux_NoVis/Tennis.x86_64"`

For instance, if you are using a Mac, then you downloaded `Tennis.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Tennis.app")
```

In [2]:
env = UnityEnvironment(file_name="Tennis_Linux/Tennis.x86_64")

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: TennisBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 8
        Number of stacked Vector Observation: 3
        Vector Action space type: continuous
        Vector Action space size (per agent): 2
        Vector Action descriptions: , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [3]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

In this environment, two agents control rackets to bounce a ball over a net. If an agent hits the ball over the net, it receives a reward of +0.1.  If an agent lets a ball hit the ground or hits the ball out of bounds, it receives a reward of -0.01.  Thus, the goal of each agent is to keep the ball in play.

The observation space consists of 8 variables corresponding to the position and velocity of the ball and racket. Two continuous actions are available, corresponding to movement toward (or away from) the net, and jumping. 

Run the code cell below to print some information about the environment.

In [4]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents 
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

Number of agents: 2
Size of each action: 2
There are 2 agents. Each observes a state with length: 24
The state for the first agent looks like: [ 0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.         -6.65278625 -1.5
 -0.          0.          6.83172083  6.         -0.          0.        ]


### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agents and receive feedback from the environment.

Once this cell is executed, you will watch the agents' performance, if they select actions at random with each time step.  A window should pop up that allows you to observe the agents.

Of course, as part of the project, you'll have to change the code so that the agents are able to use their experiences to gradually choose better actions when interacting with the environment!

In [5]:
"""
for i in range(1, 15):                                     # play game for 14 episodes
    env_info = env.reset(train_mode=False)[brain_name]     # reset the environment    
    states = env_info.vector_observations                  # get the current state (for each agent)
    scores = np.zeros(num_agents)                          # initialize the score (for each agent)
    while True:
        actions = np.random.randn(num_agents, action_size) # select an action (for each agent)
        actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
        env_info = env.step(actions)[brain_name]           # send all actions to tne environment
        next_states = env_info.vector_observations         # get next state (for each agent)
        rewards = env_info.rewards                         # get reward (for each agent)
        dones = env_info.local_done                        # see if episode finished
        scores += env_info.rewards                         # update the score (for each agent)
        states = next_states                               # roll over states to next time step
        if np.any(dones):                                  # exit loop if episode finished
            break
    print('Score (max over agents) from episode {}: {}'.format(i, np.max(scores)))
"""

"\nfor i in range(1, 15):                                     # play game for 14 episodes\n    env_info = env.reset(train_mode=False)[brain_name]     # reset the environment    \n    states = env_info.vector_observations                  # get the current state (for each agent)\n    scores = np.zeros(num_agents)                          # initialize the score (for each agent)\n    while True:\n        actions = np.random.randn(num_agents, action_size) # select an action (for each agent)\n        actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1\n        env_info = env.step(actions)[brain_name]           # send all actions to tne environment\n        next_states = env_info.vector_observations         # get next state (for each agent)\n        rewards = env_info.rewards                         # get reward (for each agent)\n        dones = env_info.local_done                        # see if episode finished\n        scores += env_info.rewards              

In [6]:
"""
#definition hyperparameters and trainingsconditions
n_episodes = 2000
max_noice= 1.0
noice_decay = 0.995
training_mode = True
random_seed = 0

#Initialize Feed-forward DNNs for Actor and Critic model 
agent = Agent(state_size*num_agents, action_size*num_agents, random_seed, max_noice, noice_decay)

current_best = 0
counter = 0
#train the agent
def ddpg_train(n_episodes):
    #list containing scores from each episode
    scores = []
    #list containing means over last 100 episodes
    means = []
    #last 100 scores
    scores_window = deque(maxlen = 100)
    # messure time for achieving a mean score over 30
    start_time = time.time()
    for episode in range(n_episodes):
        # messure time for one episode
        start_episode = time.time()
        # Reset the enviroment
        env_info = env.reset(train_mode=training_mode)[brain_name] 
        cur_states = env_info.vector_observations
        cur_states = np.reshape(cur_states,(1,state_size*num_agents))[0]
        print(cur_states)
        score = np.zeros(num_agents)
        # initialize timestep
        timestep = 0
        # reset noise
        agent.reset()
        while True:
            # Choose best action for given network
            actions = agent.act(cur_states, add_noise = True)
            actions = np.reshape(actions,(num_agents,int(len(actions)/num_agents)))
            # Action is performed and new state, reward, info are received. 
            env_info = env.step(actions)[brain_name]
            # get next state 
            next_states = env_info.vector_observations
            next_states = np.reshape(next_states,(1,state_size*num_agents))[0]
            # see if episode is finished
            dones = env_info.local_done
            # get reward
            rewards = env_info.rewards
            # save experience to replay buffer, perform learning step at defined interval
            # current state, action, reward, new state are stored in the experience replay
            agent.step(cur_states, actions, max(rewards), next_states, dones, timestep)
            # roll over new state
            cur_states = next_states      
            #add reward to score
            score += rewards
            # count timestep
            timestep+=1
            if np.any(dones):
                break
        #calculate time
        time_episode = time.time() - start_episode
        time_entire = time.time() - start_time
        #save most recent score
        scores_window.append(max(score))
        scores.append(max(score))
        mean = np.mean(scores_window)
        means.append(mean)
        if episode%100 == 0 & episode != 0:
            time_100_episodes = time.time() - start_episode
            print("Episode:" + str(episode) + " Score (max over all agents):" + str(max(score)) + 
                  " Mean Score(last 100 episodes):" + str(np.mean(scores_window)) +
                  " Duration last 100 episodes:" + time.strftime('%Mm%Ss', time.gmtime(time_100_episodes)) + 
                  " Duration training:" + time.strftime('%Hh%Mm%Ss', time.gmtime(time_entire)))
        
        #save model weights
        global current_best
        if current_best < np.mean(scores_window):
            torch.save(agent.actor_local.state_dict(), 'checkpoint_Actor.pth')
            torch.save(agent.critic_local.state_dict(), 'checkpoint_Critic.pth')
            current_best = np.mean(scores_window)
            if np.mean(scores_window) >= 0.5 and counter == 0:
                print("Environment solved in " + str(episode) + " episodes. Mean score over all 20 agents " +
                      str(np.mean(scores_window)) + " for the last 100 episodes")
                counter = 1
    return scores, means

scores, means = ddpg_train(n_episodes)

# last save
torch.save(agent.actor_local.state_dict(), 'end_weight_Actor.pth')
torch.save(agent.critic_local.state_dict(), 'end_weight_Critic.pth')

episode_scores = []
for record in scores:
    episode_scores.append(np.mean(record))

#plot the scores
fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(len(episode_scores)), episode_scores, label = "score")
plt.plot(np.arange(len(means)), means, label = "mean")
plt.axhline(y=0.5, color='r', linestyle='--', label="target")
plt.ylabel('Score')
plt.xlabel('Episode')
plt.legend()
plt.show()
"""

'\n#definition hyperparameters and trainingsconditions\nn_episodes = 2000\nmax_noice= 1.0\nnoice_decay = 0.995\ntraining_mode = True\nrandom_seed = 0\n\n#Initialize Feed-forward DNNs for Actor and Critic model \nagent = Agent(state_size*num_agents, action_size*num_agents, random_seed, max_noice, noice_decay)\n\ncurrent_best = 0\ncounter = 0\n#train the agent\ndef ddpg_train(n_episodes):\n    #list containing scores from each episode\n    scores = []\n    #list containing means over last 100 episodes\n    means = []\n    #last 100 scores\n    scores_window = deque(maxlen = 100)\n    # messure time for achieving a mean score over 30\n    start_time = time.time()\n    for episode in range(n_episodes):\n        # messure time for one episode\n        start_episode = time.time()\n        # Reset the enviroment\n        env_info = env.reset(train_mode=training_mode)[brain_name] \n        cur_states = env_info.vector_observations\n        cur_states = np.reshape(cur_states,(1,state_size*num_a

In [7]:
"""
#definition hyperparameters and trainingsconditions
n_episodes = 20000
max_noice= 1.0
noice_decay = 0.995
training_mode = True
random_seed = 0

#Initialize Feed-forward DNNs for Actor and Critic model 
agent1 = Agent(state_size, action_size, random_seed, max_noice, noice_decay)
agent2 = Agent(state_size, action_size, random_seed, max_noice, noice_decay)

current_best = 0
counter = 0
#train the agent
def ddpg_train(n_episodes):
    #list containing scores from each episode
    scores = []
    #list containing means over last 100 episodes
    means = []
    #last 100 scores
    scores_window = deque(maxlen = 100)
    # messure time for achieving a mean score over 30
    start_time = time.time()
    for episode in range(n_episodes):
        # messure time for one episode
        start_episode = time.time()
        # Reset the enviroment
        env_info = env.reset(train_mode=training_mode)[brain_name] 
        cur_states = env_info.vector_observations
        # get scores
        score = np.zeros(num_agents)
        # initialize timestep
        timestep = 0
        # reset noise
        agent1.reset()
        agent2.reset()
        while True:
            # Choose best action for given network
            actions = np.array([[0,0],[0,0]])
            actions[0] = agent1.act(cur_states[0], add_noise = True)
            actions[1] = agent2.act(cur_states[1], add_noise = True)
            # Action is performed and new state, reward, info are received. 
            env_info = env.step(actions)[brain_name]
            # get next state 
            next_states = env_info.vector_observations
            # see if episode is finished
            dones = env_info.local_done
            # get reward
            rewards = env_info.rewards
            # save experience to replay buffer, perform learning step at defined interval
            # current state, action, reward, new state are stored in the experience replay
            agent1.step(cur_states[0], actions[0], rewards[0], next_states[0], dones[0], timestep)
            agent2.step(cur_states[1], actions[1], rewards[1], next_states[1], dones[1], timestep)
            # roll over new state
            cur_states = next_states      
            #add reward to score
            score += rewards
            # count timestep
            timestep+=1
            if np.any(dones):
                break
        #calculate time
        time_episode = time.time() - start_episode
        time_entire = time.time() - start_time
        #save most recent score
        scores_window.append(max(score))
        scores.append(max(score))
        mean = np.mean(scores_window)
        means.append(mean)
        if episode%100 == 0 and episode != 0:
            time_100_episodes = time.time() - start_episode
            print("Episode:" + str(episode) + " Score (max over all agents):" + str(max(score)) + 
                    " Mean Score(last 100 episodes):" + str(np.mean(scores_window)) +
                    " Duration last episode:" + time.strftime('%Mm%Ss', time.gmtime(time_100_episodes)) + 
                    " Duration training:" + time.strftime('%Hh%Mm%Ss', time.gmtime(time_entire)))
        
        #save model weights
        global current_best
        if current_best < np.mean(scores_window):
            torch.save(agent1.actor_local.state_dict(), 'checkpoint_Actor_1.pth')
            torch.save(agent1.critic_local.state_dict(), 'checkpoint_Critic_1.pth')
            torch.save(agent1.actor_local.state_dict(), 'checkpoint_Actor_2.pth')
            torch.save(agent1.critic_local.state_dict(), 'checkpoint_Critic_2.pth')
            current_best = np.mean(scores_window)
            if np.mean(scores_window) >= 0.5 and counter == 0:
                print("Environment solved in " + str(episode) + " episodes. Mean score over all 20 agents " +
                      str(np.mean(scores_window)) + " for the last 100 episodes")
                counter = 1
    return scores, means

scores, means = ddpg_train(n_episodes)

# last save
torch.save(agent1.actor_local.state_dict(), 'end_weight_Actor_1.pth')
torch.save(agent1.critic_local.state_dict(), 'end_weight_Critic_1.pth')
torch.save(agent2.actor_local.state_dict(), 'end_weight_Actor_2.pth')
torch.save(agent2.critic_local.state_dict(), 'end_weight_Critic_2.pth')

episode_scores = []
for record in scores:
    episode_scores.append(np.mean(record))

#plot the scores
fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(len(episode_scores)), episode_scores, label = "score")
plt.plot(np.arange(len(means)), means, label = "mean")
plt.axhline(y=0.5, color='r', linestyle='--', label="target")
plt.ylabel('Score')
plt.xlabel('Episode')
plt.legend()
plt.show()
"""

'\n#definition hyperparameters and trainingsconditions\nn_episodes = 20000\nmax_noice= 1.0\nnoice_decay = 0.995\ntraining_mode = True\nrandom_seed = 0\n\n#Initialize Feed-forward DNNs for Actor and Critic model \nagent1 = Agent(state_size, action_size, random_seed, max_noice, noice_decay)\nagent2 = Agent(state_size, action_size, random_seed, max_noice, noice_decay)\n\ncurrent_best = 0\ncounter = 0\n#train the agent\ndef ddpg_train(n_episodes):\n    #list containing scores from each episode\n    scores = []\n    #list containing means over last 100 episodes\n    means = []\n    #last 100 scores\n    scores_window = deque(maxlen = 100)\n    # messure time for achieving a mean score over 30\n    start_time = time.time()\n    for episode in range(n_episodes):\n        # messure time for one episode\n        start_episode = time.time()\n        # Reset the enviroment\n        env_info = env.reset(train_mode=training_mode)[brain_name] \n        cur_states = env_info.vector_observations\n    

In [8]:
# create a new agent
agent = Agent(state_size=24, action_size=2, random_seed=1)

def maddpg(n_episodes=20000, max_t=10000, print_every=100):
    scores_deque = deque(maxlen=print_every)
    scores = []
    beta = 1.0  # factor the random noise gets multiplied with
    for i_episode in range(1, n_episodes+1):
        env_info = env.reset(train_mode=True)[brain_name]
        states = env_info.vector_observations  # get the current states
        e_scores = np.zeros(2)  # the scores of an episode for both tennis agents
        agent.reset()
        for t in range(max_t):
            if i_episode < 100:
                actions = np.random.randn(2, 2)  # use random actions for the first 100 episodes
            else:
                actions = agent.act(states, noise_factor=beta)  # let the agent select actions
            actions_other_player = np.flip(actions, 0)          # The actions of the other player
            env_info = env.step(actions)[brain_name]            # execute the selected actions and save the new information about the environment
            rewards = env_info.rewards                          # get the rewards
            next_states = env_info.vector_observations          # get the resulting states
            next_states_other_player = np.flip(next_states, 0)  # get the resulting states for the other player
            dones = env_info.local_done                         # check whether episodes have finished
            agent.step(states, actions, actions_other_player, rewards, next_states, next_states_other_player, dones)  # pass the information to the agent
            states = next_states
            e_scores += rewards
            if np.any(dones):
                break
        max_score = np.max(e_scores)  # the max score of the agents
        scores_deque.append(max_score)
        scores.append(max_score)
        beta = max(0.999 * beta, 0.01)  # reduce the noise a bit while training
        print('\rEpisode {:d}\tmax score: {:.2f}\taverage maximum score over the last 10 episodes: {:.2f}'.format(i_episode, scores_deque[-1], np.mean(list(scores_deque)[-10:])), end="")
        if i_episode % 250 == 0:
            torch.save(agent.actor_local.state_dict(), 'checkpoint_Actor.pth'.format(i_episode, scores_deque[-1]))
            torch.save(agent.critic_local.state_dict(), 'checkpoint_Critic.pth'.format(i_episode, scores_deque[-1]))
        if i_episode > 100 and np.mean(scores_deque) > 0.5:
            torch.save(agent.actor_local.state_dict(), 'checkpoint_Actor.pth')
            torch.save(agent.critic_local.state_dict(), 'checkpoint_Critic.pth')
            print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(i_episode-100, np.mean(scores_deque)))
        if i_episode % print_every == 0:
            print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_deque)))
            
    return scores

scores = maddpg()



fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(1, len(scores)+1), scores)
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.show()

Episode 100	Average Score: 0.02rage maximum score over the last 10 episodes: 0.01
Episode 200	Average Score: 0.00rage maximum score over the last 10 episodes: 0.00
Episode 300	Average Score: 0.01rage maximum score over the last 10 episodes: 0.02
Episode 400	Average Score: 0.03rage maximum score over the last 10 episodes: 0.03
Episode 500	Average Score: 0.02rage maximum score over the last 10 episodes: 0.02
Episode 600	Average Score: 0.04rage maximum score over the last 10 episodes: 0.09
Episode 700	Average Score: 0.12rage maximum score over the last 10 episodes: 0.14
Episode 800	Average Score: 0.12rage maximum score over the last 10 episodes: 0.10
Episode 900	Average Score: 0.28rage maximum score over the last 10 episodes: 0.10
Episode 972	max score: 2.60	average maximum score over the last 10 episodes: 1.43
Environment solved in 872 episodes!	Average Score: 0.50
Episode 973	max score: 0.20	average maximum score over the last 10 episodes: 1.33
Environment solved in 873 episodes!	Averag

Episode 1026	max score: 0.10	average maximum score over the last 10 episodes: 0.64
Environment solved in 926 episodes!	Average Score: 1.16
Episode 1027	max score: 0.10	average maximum score over the last 10 episodes: 0.62
Environment solved in 927 episodes!	Average Score: 1.16
Episode 1028	max score: 0.30	average maximum score over the last 10 episodes: 0.56
Environment solved in 928 episodes!	Average Score: 1.16
Episode 1029	max score: 1.00	average maximum score over the last 10 episodes: 0.40
Environment solved in 929 episodes!	Average Score: 1.17
Episode 1030	max score: 0.60	average maximum score over the last 10 episodes: 0.44
Environment solved in 930 episodes!	Average Score: 1.17
Episode 1031	max score: 0.60	average maximum score over the last 10 episodes: 0.45
Environment solved in 931 episodes!	Average Score: 1.18
Episode 1032	max score: 0.19	average maximum score over the last 10 episodes: 0.43
Environment solved in 932 episodes!	Average Score: 1.18
Episode 1033	max score: 0.5

Episode 1085	max score: 0.10	average maximum score over the last 10 episodes: 0.35
Environment solved in 985 episodes!	Average Score: 0.96
Episode 1086	max score: 0.09	average maximum score over the last 10 episodes: 0.35
Environment solved in 986 episodes!	Average Score: 0.93
Episode 1087	max score: 0.10	average maximum score over the last 10 episodes: 0.35
Environment solved in 987 episodes!	Average Score: 0.91
Episode 1088	max score: 0.10	average maximum score over the last 10 episodes: 0.35
Environment solved in 988 episodes!	Average Score: 0.88
Episode 1089	max score: 0.09	average maximum score over the last 10 episodes: 0.10
Environment solved in 989 episodes!	Average Score: 0.87
Episode 1090	max score: 0.10	average maximum score over the last 10 episodes: 0.10
Environment solved in 990 episodes!	Average Score: 0.87
Episode 1091	max score: 0.10	average maximum score over the last 10 episodes: 0.10
Environment solved in 991 episodes!	Average Score: 0.87
Episode 1092	max score: 0.1

Episode 1386	max score: 1.39	average maximum score over the last 10 episodes: 1.68
Environment solved in 1286 episodes!	Average Score: 0.75
Episode 1387	max score: 2.60	average maximum score over the last 10 episodes: 1.68
Environment solved in 1287 episodes!	Average Score: 0.78
Episode 1388	max score: 0.00	average maximum score over the last 10 episodes: 1.63
Environment solved in 1288 episodes!	Average Score: 0.78
Episode 1389	max score: 2.60	average maximum score over the last 10 episodes: 1.63
Environment solved in 1289 episodes!	Average Score: 0.80
Episode 1390	max score: 2.70	average maximum score over the last 10 episodes: 1.89
Environment solved in 1290 episodes!	Average Score: 0.83
Episode 1391	max score: 2.60	average maximum score over the last 10 episodes: 1.89
Environment solved in 1291 episodes!	Average Score: 0.85
Episode 1392	max score: 2.70	average maximum score over the last 10 episodes: 2.14
Environment solved in 1292 episodes!	Average Score: 0.88
Episode 1393	max sco

Episode 1445	max score: 0.20	average maximum score over the last 10 episodes: 0.27
Environment solved in 1345 episodes!	Average Score: 1.05
Episode 1446	max score: 0.40	average maximum score over the last 10 episodes: 0.30
Environment solved in 1346 episodes!	Average Score: 1.05
Episode 1447	max score: 2.50	average maximum score over the last 10 episodes: 0.53
Environment solved in 1347 episodes!	Average Score: 1.06
Episode 1448	max score: 0.10	average maximum score over the last 10 episodes: 0.53
Environment solved in 1348 episodes!	Average Score: 1.04
Episode 1449	max score: 0.10	average maximum score over the last 10 episodes: 0.50
Environment solved in 1349 episodes!	Average Score: 1.04
Episode 1450	max score: 0.10	average maximum score over the last 10 episodes: 0.48
Environment solved in 1350 episodes!	Average Score: 1.03
Episode 1451	max score: 0.20	average maximum score over the last 10 episodes: 0.49
Environment solved in 1351 episodes!	Average Score: 1.02
Episode 1452	max sco

...

Episode 8156	max score: 0.10	average maximum score over the last 10 episodes: 0.18
Environment solved in 8056 episodes!	Average Score: 0.59
Episode 8157	max score: 0.10	average maximum score over the last 10 episodes: 0.18
Environment solved in 8057 episodes!	Average Score: 0.58
Episode 8158	max score: 0.10	average maximum score over the last 10 episodes: 0.18
Environment solved in 8058 episodes!	Average Score: 0.58
Episode 8159	max score: 0.40	average maximum score over the last 10 episodes: 0.21
Environment solved in 8059 episodes!	Average Score: 0.58
Episode 8160	max score: 0.40	average maximum score over the last 10 episodes: 0.24
Environment solved in 8060 episodes!	Average Score: 0.58
Episode 8161	max score: 0.10	average maximum score over the last 10 episodes: 0.22
Environment solved in 8061 episodes!	Average Score: 0.58
Episode 8162	max score: 0.10	average maximum score over the last 10 episodes: 0.22
Environment solved in 8062 episodes!	Average Score: 0.56
Episode 8163	max sco

Episode 8365	max score: 0.30	average maximum score over the last 10 episodes: 0.26
Environment solved in 8265 episodes!	Average Score: 0.64
Episode 8366	max score: 0.00	average maximum score over the last 10 episodes: 0.20
Environment solved in 8266 episodes!	Average Score: 0.63
Episode 8367	max score: 0.60	average maximum score over the last 10 episodes: 0.24
Environment solved in 8267 episodes!	Average Score: 0.64
Episode 8368	max score: 0.40	average maximum score over the last 10 episodes: 0.27
Environment solved in 8268 episodes!	Average Score: 0.64
Episode 8369	max score: 0.30	average maximum score over the last 10 episodes: 0.29
Environment solved in 8269 episodes!	Average Score: 0.64
Episode 8370	max score: 1.30	average maximum score over the last 10 episodes: 0.35
Environment solved in 8270 episodes!	Average Score: 0.66
Episode 8371	max score: 0.70	average maximum score over the last 10 episodes: 0.41
Environment solved in 8271 episodes!	Average Score: 0.66
Episode 8372	max sco

Episode 8533	max score: 1.30	average maximum score over the last 10 episodes: 0.72
Environment solved in 8433 episodes!	Average Score: 0.52
Episode 8590	max score: 2.60	average maximum score over the last 10 episodes: 1.97
Environment solved in 8490 episodes!	Average Score: 0.52
Episode 8591	max score: 2.70	average maximum score over the last 10 episodes: 1.98
Environment solved in 8491 episodes!	Average Score: 0.54
Episode 8592	max score: 2.60	average maximum score over the last 10 episodes: 2.18
Environment solved in 8492 episodes!	Average Score: 0.57
Episode 8593	max score: 2.70	average maximum score over the last 10 episodes: 2.38
Environment solved in 8493 episodes!	Average Score: 0.59
Episode 8594	max score: 2.60	average maximum score over the last 10 episodes: 2.39
Environment solved in 8494 episodes!	Average Score: 0.61
Episode 8595	max score: 0.10	average maximum score over the last 10 episodes: 2.35
Environment solved in 8495 episodes!	Average Score: 0.61
Episode 8596	max sco

Episode 8648	max score: 0.40	average maximum score over the last 10 episodes: 0.66
Environment solved in 8548 episodes!	Average Score: 1.03
Episode 8649	max score: 0.70	average maximum score over the last 10 episodes: 0.70
Environment solved in 8549 episodes!	Average Score: 1.03
Episode 8650	max score: 0.80	average maximum score over the last 10 episodes: 0.57
Environment solved in 8550 episodes!	Average Score: 1.04
Episode 8651	max score: 0.70	average maximum score over the last 10 episodes: 0.61
Environment solved in 8551 episodes!	Average Score: 1.05
Episode 8652	max score: 1.40	average maximum score over the last 10 episodes: 0.73
Environment solved in 8552 episodes!	Average Score: 1.06
Episode 8653	max score: 0.40	average maximum score over the last 10 episodes: 0.72
Environment solved in 8553 episodes!	Average Score: 1.05
Episode 8654	max score: 0.20	average maximum score over the last 10 episodes: 0.70
Environment solved in 8554 episodes!	Average Score: 1.05
Episode 8655	max sco

KeyboardInterrupt: 

When finished, you can close the environment.

In [ ]:
env.close()

### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```